In [5]:
import h5py
import os
import json

from pprint import pprint

image_id = 'COCO_train2014_000000357413'
region_id = '515385'
region_anno_dir = './'
obj_ft_dir = './'

In [6]:
PIXEL_REDUCE = 1

In [7]:
idx = 0

name = '%s_%s' % (image_id, region_id)
anno = json.load(open(os.path.join(region_anno_dir, '%s.json' % image_id)))

In [8]:
anno.keys()

dict_keys(['515385', '407180', '464780', '476439', '226568'])

In [9]:
anno['515385'] # region_id

{'objects': [{'object_id': 1639311,
   'name': 'boy',
   'attributes': ['small'],
   'x': 245,
   'y': 182,
   'w': 115,
   'h': 173},
  {'object_id': 1639310,
   'name': 'grass',
   'attributes': [],
   'x': 91,
   'y': 158,
   'w': 545,
   'h': 242},
  {'object_id': 1639312,
   'name': 'frisbee',
   'attributes': [],
   'x': 286,
   'y': 236,
   'w': 123,
   'h': 75}],
 'relationships': [{'relationship_id': 867254,
   'name': 'on',
   'subject_id': 1639311,
   'object_id': 1639310},
  {'relationship_id': 867255,
   'name': 'on',
   'subject_id': 1639311,
   'object_id': 1639312}],
 'phrase': 'a small boy on some grass and a frisbee'}

In [10]:
region_graph = anno[region_id]
region_caption = anno[region_id]['phrase']

In [11]:

f = h5py.File(os.path.join(obj_ft_dir, 'train2014_%s.jpg.hdf5' % image_id), 'r')
key = 'train2014_%s.jpg' % image_id.replace('/', '_')
keys = f.keys()
print(f'key="{key}"')
obj_fts = f[key][...]

print(f[key].attrs.keys())
obj_bboxes = f[key].attrs['boxes']
obj_box_to_ft = {tuple(box): ft for box, ft in zip(obj_bboxes, obj_fts)}


key="train2014_COCO_train2014_000000357413.jpg"
<KeysViewHDF5 ['boxes', 'image_h', 'image_w']>


In [12]:
list(keys)

['train2014_COCO_train2014_000000357413.jpg']

In [13]:
mm = f[key]

In [14]:
mm[:].shape

(8, 2048)

In [15]:
# 每一个 key 对应一个 box，对应的 value 是这一个 box 区域图像的 feature
pprint(list(obj_box_to_ft.keys()))

[(0, 0, 639, 426),
 (286, 236, 408, 310),
 (255, 169, 351, 401),
 (91, 158, 635, 399),
 (339, 244, 404, 295),
 (245, 182, 404, 354),
 (245, 182, 408, 354),
 (245, 182, 359, 354)]


In [16]:
# 每一个 feature 是 2048 维的
obj_box_to_ft[(0, 0, 639, 426)].shape

(2048,)

In [17]:
PIXEL_REDUCE = 1


attn_ft, node_types, attr_order_idxs = [], [], []
obj_id_to_box = {}
for x in region_graph['objects']:
    box = (x['x'], x['y'], x['x'] + x['w'] - PIXEL_REDUCE, x['y'] + x['h'] - PIXEL_REDUCE)
    obj_id_to_box[x['object_id']] = box
    attn_ft.append(obj_box_to_ft[box])
    attr_order_idxs.append(0)
    node_types.append(0)
    for ia, attr in enumerate(x['attributes']):
        attn_ft.append(obj_box_to_ft[box])
        attr_order_idxs.append(ia + 1)
        node_types.append(1)

In [19]:
k = {'dim_hidden': 512,
 'dim_input': 2048,
 'dropout': 0.0,
 'embed_first': True,
 'freeze': False,
 'lr_mult': 1.0,
 'max_attn_len': 10,
 'num_hidden_layers': 2,
 'num_node_types': 3,
 'num_rels': 6,
 'opt_alg': 'Adam',
 'self_loop': True,
 'weight_decay': 0}
m={'attn_input_size': 512,
 'attn_size': 512,
 'attn_type': 'mlp',
 'beam_width': 1,
 'dim_word': 512,
 'dropout': 0.5,
 'fix_word_embed': False,
 'freeze': False,
 'greedy_or_beam': False,
 'hidden2word': False,
 'hidden_size': 512,
 'lr_mult': 1.0,
 'max_words_in_sent': 25,
 'memory_same_key_value': True,
 'num_layers': 1,
 'num_words': 10942,
 'opt_alg': 'Adam',
 'rnn_type': 'lstm',
 'schedule_sampling': False,
 'sent_pool_size': 1,
 'ss_increase_epoch': 5,
 'ss_increase_rate': 0.05,
 'ss_max_rate': 0.25,
 'ss_rate': 0.0,
 'tie_embed': True,
 'weight_decay': 0}
ll = {'dim_embed': 512,
 'dim_fts': [2048, 512],
 'dropout': 0,
 'freeze': False,
 'is_embed': True,
 'lr_mult': 1.0,
 'nonlinear': False,
 'norm': False,
 'opt_alg': 'Adam',
 'weight_decay': 0}

In [20]:
from itertools import chain
for k,v in chain(k.items(),m.items(),ll.items()):
    print(f"self.{k} = {v}")
    

self.dim_hidden = 512
self.dim_input = 2048
self.dropout = 0.0
self.embed_first = True
self.freeze = False
self.lr_mult = 1.0
self.max_attn_len = 10
self.num_hidden_layers = 2
self.num_node_types = 3
self.num_rels = 6
self.opt_alg = Adam
self.self_loop = True
self.weight_decay = 0
self.attn_input_size = 512
self.attn_size = 512
self.attn_type = mlp
self.beam_width = 1
self.dim_word = 512
self.dropout = 0.5
self.fix_word_embed = False
self.freeze = False
self.greedy_or_beam = False
self.hidden2word = False
self.hidden_size = 512
self.lr_mult = 1.0
self.max_words_in_sent = 25
self.memory_same_key_value = True
self.num_layers = 1
self.num_words = 10942
self.opt_alg = Adam
self.rnn_type = lstm
self.schedule_sampling = False
self.sent_pool_size = 1
self.ss_increase_epoch = 5
self.ss_increase_rate = 0.05
self.ss_max_rate = 0.25
self.ss_rate = 0.0
self.tie_embed = True
self.weight_decay = 0
self.dim_embed = 512
self.dim_fts = [2048, 512]
self.dropout = 0
self.freeze = False
self.is_embed = Tr

In [21]:

for x in region_graph['relationships']:
    obj_box = obj_id_to_box[x['object_id']]
    subj_box = obj_id_to_box[x['subject_id']]
    box = (min(obj_box[0], subj_box[0]), min(obj_box[1], subj_box[1]),
           max(obj_box[2], subj_box[2]), max(obj_box[3], subj_box[3]))
    attn_ft.append(obj_box_to_ft[box])
    node_types.append(2)
    attr_order_idxs.append(0)

num_nodes = len(node_types)
attn_ft, attn_mask = self.pad_or_trim_feature(
    np.array(attn_ft[:self.max_attn_len], np.float32),
    self.max_attn_len)
node_types = node_types[:self.max_attn_len] + [0] * max(0, self.max_attn_len - num_nodes)
node_types = np.array(node_types, np.int32)
attr_order_idxs = attr_order_idxs[:self.max_attn_len] + [0] * max(0, self.max_attn_len - num_nodes)
attr_order_idxs = np.array(attr_order_idxs, np.int32)

out = {
    'names': name,
    'mp_fts': self.mp_fts[self.img_id_to_ftidx_name[image_id][0]],
    'attn_fts': attn_ft,
    'attn_masks': attn_mask,
    'node_types': node_types,
    'attr_order_idxs': attr_order_idxs,
}
if self.is_train or self.return_label:
    sent = region_caption
    caption_ids, caption_masks = self.pad_sents(self.sent2int(sent))
    out.update({
        'caption_ids': caption_ids,
        'caption_masks': caption_masks,
        'ref_sents': [sent],
    })

NameError: name 'self' is not defined

In [18]:
anno

{'515385': {'objects': [{'object_id': 1639311,
    'name': 'boy',
    'attributes': ['small'],
    'x': 245,
    'y': 182,
    'w': 115,
    'h': 173},
   {'object_id': 1639310,
    'name': 'grass',
    'attributes': [],
    'x': 91,
    'y': 158,
    'w': 545,
    'h': 242},
   {'object_id': 1639312,
    'name': 'frisbee',
    'attributes': [],
    'x': 286,
    'y': 236,
    'w': 123,
    'h': 75}],
  'relationships': [{'relationship_id': 867254,
    'name': 'on',
    'subject_id': 1639311,
    'object_id': 1639310},
   {'relationship_id': 867255,
    'name': 'on',
    'subject_id': 1639311,
    'object_id': 1639312}],
  'phrase': 'a small boy on some grass and a frisbee'},
 '407180': {'objects': [{'object_id': 1295081,
    'name': 'boy',
    'attributes': ['young'],
    'x': 245,
    'y': 182,
    'w': 115,
    'h': 173},
   {'object_id': 1295082,
    'name': 'play',
    'attributes': [],
    'x': 0,
    'y': 0,
    'w': 640,
    'h': 427},
   {'object_id': 1295080,
    'name': 'dis